In [0]:
pip install catboost

     |████████████████████████████████| 64.7MB 61kB/s 


In [0]:
import numpy as np
import pandas as pd
import tqdm
import torch
#import catboost as cat
#from catboost import CatBoostClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
edges = pd.read_csv('gdrive/My Drive/Competitions/finnet_data/edges.csv')
ids = pd.read_csv('gdrive/My Drive/Competitions/finnet_data/ids.csv')
vertices = pd.read_csv('gdrive/My Drive/Competitions/finnet_data/vertices1.csv')
#vertices1 = pd.read_csv('gdrive/My Drive/Competitions/finnet_data/vertices.csv')
#region = pd.read_csv('gdrive/My Drive/Competitions/finnet_data/region.csv')
#region_top = pd.read_csv('gdrive/My Drive/Competitions/finnet_data/region_top.csv')

In [4]:
vertices

,id,main_okved,region_code1,company_type,524354,61537,1142564,300432,83777,206473,58408,776150,1292407,361401,1301544,1203323,1209828,341670,83818,1189202,1434836,929264,1263393,1342003,1063763,41216,1227912,983180,324065,258315,1160709,1443535,214507,1319172,1147550,785334,640944,1154568,1427428,409925,...,310694,732829,386565,407136,319548,1471693,1517903,244691,598328,550468,55389,966969,47929,1439287,936788,995150,1025839,1202376,89473,1523148,1036225,457698,1024638,811809,642175,1392199,1234038,912470,1526265,52803,373737,1244877,main_okved1,region_code,amount,am_per,top,population,main_okved2,edges
0,1,7.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.75,77,213973,17,1,12678079,47.0,2.0
1,2,6.0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.20,78,42132,8,4,5398064,41.0,2.0
2,3,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25.11,50,124818,17,2,7690863,25.0,1.0
3,4,7.0,6,1,0,0,0,0,1,0,0,0,1,1,1,0,0,1,0,1,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,45.31,89,2500,4,71,544444,45.0,4.0
4,5,9.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56.10,50,124818,17,2,7690863,56.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534744,1534745,10.0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63.99,77,213973,17,1,12678079,64.0,3.0
1534745,1534746,7.0,6,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,47.19,66,27512,6,5,4310681,47.0,1.0
1534746,1534747,6.0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.20,77,213973,17,1,12678079,41.0,2.0
1534747,1534748,13.0,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,74.20,33,5676,4,34,1358416,74.0,12.0


In [0]:
'''vertices = vertices.rename(columns={'region_code1':'region_code', 'region_code':'region_code1'})
vertices = vertices.merge(region, on=['region_code'])
vertices = vertices.sort_values(by = 'id').reset_index(drop=True)'''

In [0]:
vertices = vertices.merge(region_top, on=['region_code'])
vertices = vertices.sort_values(by = 'id').reset_index(drop=True)

In [0]:
vertices

,id,main_okved,region_code1,company_type,524354,61537,1142564,300432,83777,206473,58408,776150,1292407,361401,1301544,1203323,1209828,341670,83818,1189202,1434836,929264,1263393,1342003,1063763,41216,1227912,983180,324065,258315,1160709,1443535,214507,1319172,1147550,785334,640944,1154568,1427428,409925,...,1331069,310694,732829,386565,407136,319548,1471693,1517903,244691,598328,550468,55389,966969,47929,1439287,936788,995150,1025839,1202376,89473,1523148,1036225,457698,1024638,811809,642175,1392199,1234038,912470,1526265,52803,373737,1244877,mean,main_okved1,region_code,amount,am_per,top,population
0,1,7.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,186953.962013,46.75,77,213973,17,1,12678079
1,2,6.0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121195.806420,41.20,78,42132,8,4,5398064
2,3,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1152.274902,25.11,50,124818,17,2,7690863
3,4,7.0,6,1,0,0,0,0,1,0,0,0,1,1,1,0,0,1,0,1,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,113666.349206,45.31,89,2500,4,71,544444
4,5,9.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,85019.232398,56.10,50,124818,17,2,7690863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534744,1534745,10.0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121050.770582,63.99,77,213973,17,1,12678079
1534745,1534746,7.0,6,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1211.607937,47.19,66,27512,6,5,4310681
1534746,1534747,6.0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,98274.098334,41.20,77,213973,17,1,12678079
1534747,1534748,13.0,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120077.744632,74.20,33,5676,4,34,1358416


In [0]:
vertices1['main_okved']

0          46.75
1          41.20
2          25.11
3          45.31
4          56.10
           ...  
1534744    63.99
1534745    47.19
1534746    41.20
1534747    74.20
1534748    68.20
Name: main_okved, Length: 1534749, dtype: float64

In [0]:
vertices1 = vertices1.rename(columns={'main_okved':'main_okved1'})

In [0]:
vertices = vertices.join(vertices1['main_okved1'])

In [0]:
vertices1 = vertices1.rename(columns={'region_code':'region_code1'})
vertices = vertices.join(vertices1['region_code1'])

In [0]:
#vertices.company_type.unique()

array([1, 0, 2, 3, 4, 5])

In [0]:
'''target_mapping = {
    'Unknown': 0,
    'Limited': 1,
    'Actionary': 2,
    'NonComm': 3,
    'Individual': 4, 
    'Agri': 5
}
vertices.company_type = vertices.company_type.replace(target_mapping)'''

"target_mapping = {\n    'Unknown': 0,\n    'Limited': 1,\n    'Actionary': 2,\n    'NonComm': 3,\n    'Individual': 4, \n    'Agri': 5\n}\nvertices.company_type = vertices.company_type.replace(target_mapping)"

In [0]:
vertices.drop(vertices.columns[0], axis=1, inplace=True)

In [0]:
vertices.head()

,id,main_okved,region_code,company_type,524354,61537,1142564,300432,83777,206473,58408,776150,1292407,361401,1301544,1203323,1209828,341670,83818,1189202,1434836,929264,1263393,1342003,1063763,41216,1227912,983180,324065,258315,1160709,1443535,214507,1319172,1147550,785334,640944,1154568,1427428,409925,...,51755,1500415,596221,641486,1331069,310694,732829,386565,407136,319548,1471693,1517903,244691,598328,550468,55389,966969,47929,1439287,936788,995150,1025839,1202376,89473,1523148,1036225,457698,1024638,811809,642175,1392199,1234038,912470,1526265,52803,373737,1244877,mean,main_okved1,region_code1
0,1,7.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,186953.962013,46.75,77
1,2,6.0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121195.806420,41.20,78
2,3,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1152.274902,25.11,50
3,4,7.0,6,1,0,0,0,0,1,0,0,0,1,1,1,0,0,1,0,1,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,113666.349206,45.31,89
4,5,9.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,85019.232398,56.10,50


In [0]:
np.random.seed(42)

In [0]:
'''for i in tqdm.tqdm(ids.id):
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    c = []
    vertices[str(i)] = 0
    a = list(edges[edges['id_1']==i]['id_2'].unique())
    b = list(edges[edges['id_2']==i]['id_1'].unique())
    a.extend(b)

    for j in a:
      a1 = list(edges[edges['id_1']==j]['id_2'].unique())
      b1 = list(edges[edges['id_2']==j]['id_1'].unique())
      a1.extend(b1)
      c.extend(a1)
    a1 = []
    b1 = []
    a = []
    b = [] 
    for k in c:
      vertices.loc[k-1, str(i)] += 1'''

100%|██████████| 100/100 [10:07:02<00:00, 364.23s/it]


In [0]:
vertices

,id,main_okved,region_code1,company_type,524354,61537,1142564,300432,83777,206473,58408,776150,1292407,361401,1301544,1203323,1209828,341670,83818,1189202,1434836,929264,1263393,1342003,1063763,41216,1227912,983180,324065,258315,1160709,1443535,214507,1319172,1147550,785334,640944,1154568,1427428,409925,...,386565,407136,319548,1471693,1517903,244691,598328,550468,55389,966969,47929,1439287,936788,995150,1025839,1202376,89473,1523148,1036225,457698,1024638,811809,642175,1392199,1234038,912470,1526265,52803,373737,1244877,main_okved1,region_code,amount,am_per,top,population,main_okved2,value,n_transactions,mean
0,1,7.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.75,77,213973,17,1,12678079,47.0,7.478158e+05,2.000000,373907.924027
1,2,6.0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.20,78,42132,8,4,5398064,41.0,9.293199e+05,187.749487,4949.786376
2,3,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25.11,50,124818,17,2,7690863,25.0,4.788663e+05,207.791701,2304.549804
3,4,7.0,6,1,0,0,0,0,1,0,0,0,1,1,1,0,0,1,0,1,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,45.31,89,2500,4,71,544444,45.0,1.361190e+06,604.608179,2251.359326
4,5,9.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56.10,50,124818,17,2,7690863,56.0,1.578424e+06,535.494874,2947.598499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534744,1534745,10.0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63.99,77,213973,17,1,12678079,64.0,1.142074e+06,162.995900,7006.765846
1534745,1534746,7.0,6,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,47.19,66,27512,6,5,4310681,47.0,5.035241e+05,207.791701,2423.215874
1534746,1534747,6.0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.20,77,213973,17,1,12678079,41.0,7.906844e+05,81.997950,9642.733637
1534747,1534748,13.0,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,74.20,33,5676,4,34,1358416,74.0,4.597791e+06,618.781450,7430.396243


In [0]:
vertices.main_okved.unique()

array([ 7.,  6.,  3.,  9., 13., 17., 12., 10., 19.,  5.,  8., 16., 18.,
       14.,  1., 15., 11.,  4.,  2., 20., 21.])

In [0]:
edges

,id_1,id_2,value,n_transactions,mean
0,878327,1133997,478035.238733,277.747437,1721.114850
1,707356,1341541,442189.669684,80.997950,5459.269900
2,169982,494074,353097.929209,287.789650,1226.930603
3,551010,979933,537749.674840,426.743337,1260.124361
4,76064,597023,418990.198382,287.789650,1455.890432
...,...,...,...,...,...
4685639,533291,1134794,275822.479741,1.000000,275822.479741
4685640,929970,1378868,507684.088750,1.000000,507684.088750
4685641,45803,829031,318785.728608,1.000000,318785.728608
4685642,1266550,1448478,494106.825312,1.000000,494106.825312


In [0]:
edges.groupby('id_1').count()['id_2']

id_1
1          2
2          2
3          1
4          4
5          4
          ..
1532726    1
1533068    1
1533321    1
1534041    1
1534210    1
Name: id_2, Length: 1017864, dtype: int64

In [0]:
v = edges.groupby('id_1').count()['id_2']
k = edges.groupby('id_2').count()['id_1']
vertices.index += 1
vertices = vertices.join(v).fillna(0)
vertices = vertices.join(k).fillna(0)
vertices.index -= 1
vertices['edges'] = vertices['id_1'] + vertices['id_2']
vertices.drop(vertices.columns[[111, 112]], axis=1, inplace=True)

In [0]:
vertices

,id,main_okved,region_code1,company_type,524354,61537,1142564,300432,83777,206473,58408,776150,1292407,361401,1301544,1203323,1209828,341670,83818,1189202,1434836,929264,1263393,1342003,1063763,41216,1227912,983180,324065,258315,1160709,1443535,214507,1319172,1147550,785334,640944,1154568,1427428,409925,...,310694,732829,386565,407136,319548,1471693,1517903,244691,598328,550468,55389,966969,47929,1439287,936788,995150,1025839,1202376,89473,1523148,1036225,457698,1024638,811809,642175,1392199,1234038,912470,1526265,52803,373737,1244877,main_okved1,region_code,amount,am_per,top,population,main_okved2,edges
0,1,7.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.75,77,213973,17,1,12678079,47.0,2.0
1,2,6.0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.20,78,42132,8,4,5398064,41.0,2.0
2,3,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25.11,50,124818,17,2,7690863,25.0,1.0
3,4,7.0,6,1,0,0,0,0,1,0,0,0,1,1,1,0,0,1,0,1,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,45.31,89,2500,4,71,544444,45.0,4.0
4,5,9.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56.10,50,124818,17,2,7690863,56.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534744,1534745,10.0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63.99,77,213973,17,1,12678079,64.0,3.0
1534745,1534746,7.0,6,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,47.19,66,27512,6,5,4310681,47.0,1.0
1534746,1534747,6.0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.20,77,213973,17,1,12678079,41.0,2.0
1534747,1534748,13.0,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,74.20,33,5676,4,34,1358416,74.0,12.0


In [0]:
vertices.shape

(1534749, 105)

In [0]:
vertices.region_code.unique()

array([1, 2, 6, 3, 5, 8, 7, 4, 0])

In [0]:
'''for i in range(1534749):
  if vertices.region_code[i] in [31, 32, 33, 36, 37, 40, 44, 46, 48, 50, 90, 57, 62, 67, 68, 69, 71, 76, 77, 97, 99]:
    vertices.region_code[i] = 1
  elif vertices.region_code[i] in [10, 11, 29, 35, 39, 91, 47, 51, 53, 60, 83, 78, 98]:
    vertices.region_code[i] = 2
  elif vertices.region_code[i] in [1, 8, 82, 23, 93, 123, 30, 34, 61, 92]:
    vertices.region_code[i] = 3
  elif vertices.region_code[i] in [5, 6, 7, 9, 15, 20, 95, 26]:
    vertices.region_code[i] = 4
  elif vertices.region_code[i] in [12, 13, 16, 18, 21, 59, 81, 43, 52, 56, 58, 63, 64, 73]:
    vertices.region_code[i] = 5
  elif vertices.region_code[i] in [45, 66, 96, 72, 74, 86, 89]:
    vertices.region_code[i] = 6
  elif vertices.region_code[i] in [17, 22, 4, 19, 24, 84, 88, 38, 85, 42, 54, 55, 70]:
    vertices.region_code[i] = 7
  elif vertices.region_code[i] in [3, 14, 41, 75, 80, 25, 27, 28, 49, 65, 79, 87]:
    vertices.region_code[i] = 8'''

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/loc

In [0]:
'''vertices.loc[(vertices.main_okved<5), 'main_okved'] = 1
vertices.loc[((vertices.main_okved>=5) & (vertices.main_okved<10)), 'main_okved'] = 2
vertices.loc[((vertices.main_okved>=10) & (vertices.main_okved<35)), 'main_okved'] = 3
vertices.loc[((vertices.main_okved>=35) & (vertices.main_okved<36)), 'main_okved'] = 4
vertices.loc[((vertices.main_okved>=36) & (vertices.main_okved<41)), 'main_okved'] = 5
vertices.loc[((vertices.main_okved>=41) & (vertices.main_okved<45)), 'main_okved'] = 6
vertices.loc[((vertices.main_okved>=45) & (vertices.main_okved<49)), 'main_okved'] = 7
vertices.loc[((vertices.main_okved>=49) & (vertices.main_okved<55)), 'main_okved'] = 8
vertices.loc[((vertices.main_okved>=55) & (vertices.main_okved<58)), 'main_okved'] = 9
vertices.loc[((vertices.main_okved>=58) & (vertices.main_okved<64)), 'main_okved'] = 10
vertices.loc[((vertices.main_okved>=64) & (vertices.main_okved<68)), 'main_okved'] = 11
vertices.loc[((vertices.main_okved>=68) & (vertices.main_okved<69)), 'main_okved'] = 12
vertices.loc[((vertices.main_okved>=69) & (vertices.main_okved<77)), 'main_okved'] = 13
vertices.loc[((vertices.main_okved>=77) & (vertices.main_okved<84)), 'main_okved'] = 14
vertices.loc[((vertices.main_okved>=84) & (vertices.main_okved<85)), 'main_okved'] = 15
vertices.loc[((vertices.main_okved>=85) & (vertices.main_okved<86)), 'main_okved'] = 16
vertices.loc[((vertices.main_okved>=86) & (vertices.main_okved<90)), 'main_okved'] = 17
vertices.loc[((vertices.main_okved>=90) & (vertices.main_okved<94)), 'main_okved'] = 18
vertices.loc[((vertices.main_okved>=94) & (vertices.main_okved<97)), 'main_okved'] = 19
vertices.loc[((vertices.main_okved>=97) & (vertices.main_okved<99)), 'main_okved'] = 20
vertices.loc[(vertices.main_okved>=99), 'main_okved'] = 21'''

In [0]:
vertices.to_csv('gdrive/My Drive/Competitions/finnet_data/vertices1.csv', index = False)

In [0]:
vertices['main_okved2'] = vertices['main_okved1'].round()

In [0]:
vertices['mean'] = vertices['value']/vertices['n_transactions']

In [0]:
vertices[['n_transactions', 'value']].corr()

,n_transactions,value
n_transactions,1.000000,0.857844
value,0.857844,1.000000


In [0]:
edges.corr()

,id_1,id_2,value,n_transactions
id_1,1.000000,0.500048,-0.013452,-0.006768
id_2,0.500048,1.000000,-0.016872,-0.007840
value,-0.013452,-0.016872,1.000000,0.567790
n_transactions,-0.006768,-0.007840,0.567790,1.000000


In [0]:
result = pd.DataFrame(columns=['id_1', 'id_2', 'preds'])

In [0]:
#model = RandomForestClassifier(n_estimators=5, random_state=9, n_jobs=-1, max_depth=10)
#model = KNeighborsClassifier(n_neighbors=100, n_jobs=4)
#model = AdaBoostClassifier(n_estimators=50, random_state=9, learning_rate=0.8)
model = GradientBoostingClassifier(n_estimators=5, random_state=9)

In [0]:
k = np.random.randint(1, 1534749, 50)

In [6]:
# для каждой вершины из ids с помощью catboost найдем 1000 самых вероятных ребер
for i in tqdm.tqdm(ids.id):
    # соберем датасет из всех возможных вершин
    # вершины имеющие в исходных данных ребро с i обозначим 1, остальные 0
    # учтем то, что вершина i может быть как среди id_1, так и среди id_2
    #vertices1 = vertices[vertices[str(i)]!=0]
    #vertices2 = vertices[vertices[str(i)]==0].sample(frac=1).iloc[:vertices1.shape[0]*5]
    #vertices3 = vertices1.append(vertices2).sample(frac=1).reset_index(drop=True)
    df1 = edges[edges['id_1'] == i].reset_index()
    
    df2 = edges[edges['id_2'] == i].reset_index()

    df = df1[['id_2', 'id_1']].rename(columns={'id_1':'id_2', 'id_2':'id_1'}).append(df2[['id_1', 'id_2']])
    df['target'] = 1

    df = vertices.set_index('id').join(df.set_index('id_1')['target']).fillna(0)

    X = df[['main_okved', 'region_code', 'company_type', str(i), 'main_okved2', 'region_code1', 'main_okved1', 'amount', 'population', 'top', 'am_per']]#, 'company_type', str(i), 'mean']]df[['main_okved', , 'main_okved2' 'population', 'top' 
    y = df['target']
    
    # создание модели с указанием гиперпараметра C
    # обучение модели
    # предсказание на тестовой выборке
    model.fit(X, y)
    preds = model.predict_proba(X)[:,1]
    
    df['preds'] = preds
    df['id_2'] = i
    # возьмем первую 1000 предсказанных ребер, исключив те, про которые мы уже знали
    res = df[df['target'] != 1].reset_index()[['id', 'id_2', 'preds']]

    res.columns = ['id_1', 'id_2', 'preds']

    result = result.append(res, ignore_index=True, sort=False)
    if result.shape[0] > 110000:
      result = result.sort_values(by='preds', ascending=False).iloc[:110000]

100%|██████████| 100/100 [39:18<00:00, 23.59s/it]


In [0]:
#train_set, val_set = 

In [24]:
model.feature_importances_

array([0.  , 0.14, 0.04, 0.04, 0.  , 0.1 , 0.46, 0.08, 0.06, 0.08, 0.  ])

In [0]:
model.feature_importances_

array([0.04052763, 0.05985851, 0.04388605, 0.09032062, 0.08935303,
       0.03098908, 0.20017916, 0.04699563, 0.04868474, 0.05324395,
       0.02456357, 0.27139803])

In [25]:
result.head(100000)

,id_1,id_2,preds
0,154440,676805,0.528729
1,1526265,1526265,0.525158
2,419415,1526265,0.525158
3,1071144,1526265,0.525158
4,676805,676805,0.524656
...,...,...,...
99027,699835,361401,0.480344
99028,875139,361401,0.480344
99029,765495,713665,0.480343
99030,1291104,1160709,0.480335


In [0]:
result1 = result.copy(deep=True)

In [0]:
result1 = result1[['id_1', 'id_2']]

In [28]:
result1.id_2.nunique()

90

In [29]:
result1[pd.DataFrame(np.sort(result1.values), columns=result1.columns, index=result1.index).duplicated(keep=False)]

,id_1,id_2
37,1036225,1342003
38,676805,1526265
40,1052575,732829
45,929264,258675
54,929264,732829
...,...,...
99037,732829,244691
107352,1526265,676805
108856,457698,361401
108989,1356407,929264


In [0]:
result1.drop(result1[pd.DataFrame(np.sort(result1.values), columns=result1.columns, index=result1.index).duplicated(keep='first')].index, inplace=True)

In [31]:
result1[pd.DataFrame(np.sort(result1.values), columns=result1.columns, index=result1.index).duplicated(keep=False)]

,id_1,id_2


In [0]:
submission = result1.iloc[:100000]

In [33]:
submission['id_2'].nunique()

89

In [34]:
submission.groupby('id_2')['id_1'].nunique().min()

2

In [35]:
submission

,id_1,id_2
0,154440,676805
1,1526265,1526265
2,419415,1526265
3,1071144,1526265
4,676805,676805
...,...,...
99140,1299194,1147550
99141,568818,1147550
99142,971097,1147550
99143,667752,1147550


In [0]:
submission.to_csv('gdrive/My Drive/Competitions/finnet_data/submission.csv', index = False)